## Librerias

In [4]:
import requests
import time
from datetime import datetime
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
from selenium import webdriver
import pprint
import pandas as pd
import names as nm
import numpy as np
import traceback
import datetime
import re
import pandas as pd

# Funciones 

##  Selenium

In [5]:
def click(buscador, driver, atributo="xpath"):
    """
    atributos : xpath, name, css, class, id
    """
    if atributo=="xpath":
        "dado un full xpath hace click"
        elemento  = WebDriverWait(driver, 15).until( \
            EC.presence_of_element_located((By.XPATH, f"{buscador}")))
    elif atributo=="name":
        elemento  = WebDriverWait(driver, 15).until( \
            EC.presence_of_element_located((By.NAME, f"{buscador}")))   
    
    elif atributo=="css":
        elemento  = WebDriverWait(driver, 15).until( \
            EC.presence_of_element_located((By.CSS_SELECTOR, f"{buscador}")))
    
    elif atributo=="class":
         elemento  = WebDriverWait(driver, 15).until( \
            EC.presence_of_element_located((By.CLASS_NAME, f"{buscador}")))
    elif atributo=="id":
         elemento  = WebDriverWait(driver, 15).until( \
            EC.presence_of_element_located((By.ID, f"{buscador}")))
    
        
    time.sleep(0.8)
    elemento.click()
    return elemento

def select_option(xpath_click, xpath_write, name, driver):
    """Selecciona el DROPDOWN, luego ESCRIBE name, FInalemente
    selecciona la primera opción, el xpath_write"""
    
    elemento  = WebDriverWait(driver, 15).until( \
    EC.presence_of_element_located((By.XPATH, f"{xpath_click}")))
    time.sleep(1)
    elemento.click()
    time.sleep(2)
    elemento2  = WebDriverWait(driver, 15).until( \
        EC.presence_of_element_located((By.XPATH, f"{xpath_write}")))
    time.sleep(2)
    elemento2.click()
    slow_type(elemento2, name)
    elemento2.send_keys(Keys.ENTER)


def slow_type(element, text, delay=0.3):
    """Send a text to an element one character at a time with a delay."""
    for character in text:
        element.send_keys(character)
        time.sleep(delay+ np.max(np.random.normal(0.15,0.1),0) ) 
    

def escribir_y_enter(xpath, driver, texto="",sleep_time=0.8, atributo="xpath"):
    """
    atributo : xpath, name, css, class, id
    """
    elemento = click(xpath,driver, atributo=atributo)
    time.sleep(sleep_time)
    slow_type(elemento, texto)
    elemento.send_keys(Keys.ENTER)

### BS 

In [6]:
def soup_of_page(driver):
    time.sleep(2.5)
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    return soup

### Algoritmos 

In [7]:
def chequera_indice_provincia(html_tags, key):
    '''busca la provincia key en el array de html tags
    y devuelve el indice'''
    prov = key.strip().capitalize()
    for idx, provi in  enumerate(html_tags):
        res  = provi.get_text()
        res = str(res)
        res = res.strip().capitalize()
        if res == prov:
            return idx+1
    return "no encontrado"

### Generales

In [8]:
def norm_texto(string):
    nombre = str(string)
    nombre = nombre.strip().lower()
    return nombre


def generar_persona():
    '''return 
    sexo, nombre, apellido, email, numero'''
    data = {}
    #nombre y apeelido
    sexo =np.random.binomial(n=1, p=0.5)
    sexo = np.where(sexo==0, "male", "female" )
    random_number= np.random.randint(0,9999, 1)[0]
    nombre = nm.get_first_name(gender=sexo)
    apellido =  nm.get_last_name()
    fin_mail= "@gmail.com"
    email = nombre+apellido+str(random_number)+fin_mail
    email = email.lower()
    numero = np.random.randint(0,9,size=8)
    numero_str = ''.join([str(elem) for elem in numero])
    data["sexo"] = sexo
    data["nombre"] = nombre
    data["apellido"] = apellido
    data["email"] = email
    data ["numero" ] = numero_str
    return data

def calcular_edad(nacimiento_str):
    nacimiento_str = str(nacimiento_str)
    fecha =  datetime.datetime.strptime(nacimiento_str, '%d%m%Y').date()
    diff = datetime.date.today() - fecha
    edad = round(diff.days/365)
    return edad

## Comparar en Casa 

### Parse URLS

In [9]:
def precios_y_suma(driver, cobertura = "Todo Riesgo", _id=1):
    soup = soup_of_page(driver)

    tags = soup.find_all("img", {"class":"company-logo"})
    companias = [x.attrs["alt"] for x in tags if len(x.attrs["alt"])>2]

    errores = 0
    data = {}
    df = pd.DataFrame()


    for idx,compania  in enumerate(companias):

        idx += 3
        try:
            if idx==3:
                driver.set_window_size(1920, 1080)
                url = "/html/body/div[2]/div[2]/div/div/div/div[3]/div[1]/div[2]/div/div[22]/div"
            else:
                url =f"/html/body/div[2]/div[2]/div/div/div/div[3]/div[{idx}]/div/div/div[8]/div[1]/div[1]"
            click(url, driver)
            #Precio
            driver.set_window_size(929, 1080)
            soup = soup_of_page(driver)
            precio = soup.find_all("p", {"class":"tc"})[0].span.get_text()
            suma_asegurada = soup.find_all("div", {"class":"col col-xs-5 col-md-4 text-right"})[0].get_text()
            suma_asegurada = suma_asegurada.split("$")[1].strip()
            df.loc["precio",f"{compania}"] = precio
            df.loc["suma_asegurada", f"{compania}"] = suma_asegurada
            url = "/html/body/div[2]/div[2]/div/div/div/div[3]/div[35]/div/div[1]"
            click(url, driver)
        except:
            print("fallo con compania: ",compania)
            errores+=1
            if errores>=3:
                break
    #momento actual
    df["id"] = _id
    df["cobertura"] = cobertura
    df["Fecha"] = datetime.date.today().strftime("%d-%m-%Y")

    #Reordeno columnas
    cols = list(df.columns)
    cols2= cols[-3:] + cols[:-3]
    df = df[cols2]
    df.index.name = 'conceptos'
    data = pd.read_excel(r"./data_perfiles.xlsx", index_col="conceptos")
    resultado =pd.concat([data ,df])
    resultado = resultado.drop_duplicates()
    resultado = resultado.fillna("nan")
    resultado.to_excel("data_perfiles.xlsx")

###  Get URLS

In [52]:

def get_url_perfil(marca, ano_vehiculo, modelo, version, codigo, edad, sexo, driver):




    #continuar
        
    url_base = "https://www.comparaencasa.com/seguros-de-auto/cotizador/?"
    marcas =[('volkswagen', '157'),
     ('renault', '130'),
     ('peugeot', '119'),
     ('chevrolet', '28'),
     ('ford', '53'),
     ('fiat', '50'),
     ('citroen', '32'),
     ('toyota', '153'),
     ('honda', '70'),
     ('acura', '1'),
     ('aeolus', '2'),
     ('aleko', '4'),
     ('alfa romeo', '5'),
     ('aro', '7'),
     ('asia', '8'),
     ('audi', '11'),
     ('austin', '12'),
     ('autobianchi', '13'),
     ('belavtomaz', '14'),
     ('bertone', '15'),
     ('blac', '16'),
     ('bmw', '17'),
     ('chery', '27'),
     ('chrysler', '31'),
     ('dodge', '31'),
     ('jeep', '31'),
     ('dacia', '35'),
     ('daewoo', '36'),
     ('daihatsu', '37'),
     ('deutz', '38'),
     ('dimex', '39'),
     ('eniak', '44'),
     ('f.e.r.e.s.a.', '46'),
     ('ferrari', '49'),
     ('g.a.z.', '56'),
     ('grosspal', '63'),
     ('ham-jiang', '66'),
     ('heibao', '67'),
     ('hino', '68'),
     ('hummer', '71'),
     ('hyundai', '72'),
     ('international', '74'),
     ('isuzu', '75'),
     ('iveco', '77'),
     ('izh', '79'),
     ('jac', '80'),
     ('jaguar', '82'),
     ('jinbei', '84'),
     ('kamaz', '86'),
     ('kenworth', '87'),
     ('kia', '88'),
     ('lada', '90'),
     ('lancia', '92'),
     ('liaz', '95'),
     ('lifan', '307'),
     ('maestro', '100'),
     ('mahindra', '101'),
     ('maserati', '102'),
     ('mazda', '104'),
     ('mercedes benz', '105'),
     ('metro', '109'),
     ('mini cooper', '111'),
     ('mitsubishi', '112'),
     ('nakai', '115'),
     ('nissan', '116'),
     ('opel', '118'),
     ('piaggio', '121'),
     ('polonez', '123'),
     ('pontiac', '124'),
     ('porsche', '125'),
     ('proton', '126'),
     ('ranquel', '128'),
     ('rolls royce', '132'),
     ('rover', '133'),
     ('saab', '134'),
     ('santana', '135'),
     ('sanxing', '136'),
     ('scania', '138'),
     ('seat', '139'),
     ('skoda', '142'),
     ('space', '144'),
     ('star', '147'),
     ('subaru', '148'),
     ('suzuki', '149'),
     ('tata', '151'),
     ('tavria', '152'),
     ('uaz', '156'),
     ('volvo', '160'),
     ('wuling motors', '164'),
     ('yantai', '165'),
     ('yuejin', '166')]
    resultado = [x for x in marcas if x[0] == norm_texto(marca)][0]
    _id = resultado[1]
    _id
    url_continuado =  f"car_brand={_id}&year={ano_vehiculo}"

    url = url_base+url_continuado
    driver.get(url)
   
    #MODELO
    titulo = driver.find_element_by_xpath("/html/body/div[1]/div[3]/div/div/div/h2/b/span")
    titulo.click()
    time.sleep(2.5)
    path_1 = "/html/body/div[1]/div[3]/div/div/form/div[1]/div[2]/div[1]/div/div[1]/a"
    path_2 = "/html/body/div[1]/div[3]/div/div/form/div[1]/div[2]/div[1]/div/div[1]/div/div[1]/input"
    select_option(path_1, path_2, modelo,driver)
    time.sleep(3)
        
        
    soup = soup_of_page(driver)

    ##########AGREGAR OTRO METODO DE SELECION POR NOMBRE########################## y un try catch
    #VERSION
    soup = soup_of_page(driver)
    sel = soup.find("select", {"name":'car_version' })
    opt = [{x.get_text(): x.get_attribute_list("value")[0]} for x in sel.find_all("option") if len(x.get_text().strip())>2]
    print(opt)
    print(version)
    posta = [x for x in opt if str(version).upper() in re.sub(' +', ' ',  str(x))][0]
    valor = list(posta.values())[0]
    print(valor)
    driver.quit()

    
    get_url = url + f"&car_version={str(valor)}&car_model={str(modelo).upper()}"
    time.sleep(15)
    t= 35+ max( np.random.normal(26,24),0)
    print("tiempo de espera: ",t )
    time.sleep(t) 
    profile = webdriver.FirefoxProfile(r'C:\Users\Matias\Desktop\Tor Browser\Browser\TorBrowser\Data\Browser\profile.default')
    profile.set_preference('network.proxy.type', 1)
    profile.set_preference('network.proxy.socks', '127.0.0.1')
    profile.set_preference('network.proxy.socks_port', 9150)
    profile.set_preference("network.proxy.socks_remote_dns", False)
    profile.update_preferences()
    firefox_options = webdriver.FirefoxOptions()
    firefox_options.binary_location = r'C:\Program Files\Mozilla Firefox\firefox.exe'
    options = Options()
    options.add_argument("--window-size=1980,1080")
    options.add_argument("start-maximized")
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    driver = webdriver.Firefox(firefox_profile= profile, options = firefox_options, executable_path=r'./geckodriver.exe')
    driver.get(get_url)

        
    time.sleep(12)

    #0KM
    #poor default es nuevo
    try:
        click("/html/body/div[1]/div[3]/div/div/form/div[1]/div[3]/div/div[1]/label/div", driver)
    except:
        print("")
    
    time.sleep(1.25)
    #GNC
    #no por default
    path_no= "/html/body/div[1]/div[3]/div/div/form/div[1]/div[4]/div/div/label[2]/div"
    click(path_no,driver)
    time.sleep(1.25)

    #Codigo postal
    escribir_y_enter("id_zip_code", driver,texto=codigo, atributo="id")

    time.sleep(1.35)
    #Edad
    escribir_y_enter("age", driver, texto=edad, atributo="name")
    time.sleep(1.35)


    #Genero
    if sexo=="hombre":
        click("/html/body/div[1]/div[3]/div/div/form/div[1]/div[6]/div[2]/div/label[1]/div", driver)
    else:
        click("/html/body/div[1]/div[3]/div/div/form/div[1]/div[6]/div[2]/div/label[2]/div", driver)
    time.sleep(2)

    try:
        persona = generar_persona()
        escribir_y_enter("email", driver, texto=persona["email"], atributo="name")
    except:
        print("")
    time.sleep(3)
    
    try:
        click("btn btn-success btn-lg btn-form submit", driver, atributo="class")
    except:
        print("")
        
    time.sleep(15)
    url = driver.current_url
    time.sleep(2)
    driver.quit()

               
    return url


In [53]:
def get_all_perfiles():
    perfiles = pd.read_excel("./perfiles.xlsx")
    tiempo_por_perfil = 40
    cantidad_periles= perfiles.shape[0]
    tiempo_estimado = tiempo_por_perfil*cantidad_periles 

    urls = {}
    print(f"Tiempo estimado total es: {round(tiempo_estimado/60,2)}, minutos")
    for i in range(perfiles.shape[0]):
        perfil = dict(perfiles.iloc[i])
        perfil["edad"] = calcular_edad(perfil["fecha (edad)"])
        print(f"*{perfil['id']} - ID, Obteniendo el perfil")
        profile = webdriver.FirefoxProfile(r'C:\Users\Matias\Desktop\Tor Browser\Browser\TorBrowser\Data\Browser\profile.default')
        profile.set_preference('network.proxy.type', 1)
        profile.set_preference('network.proxy.socks', '127.0.0.1')
        profile.set_preference('network.proxy.socks_port', 9150)
        profile.set_preference("network.proxy.socks_remote_dns", False)
        profile.update_preferences()
        firefox_options = webdriver.FirefoxOptions()
        firefox_options.binary_location = r'C:\Program Files\Mozilla Firefox\firefox.exe'
        options = Options()
        options.add_argument("--window-size=1980,1080")
        options.add_argument("start-maximized")
        #options.add_experimental_option("excludeSwitches", ["enable-automation"])
        #options.add_experimental_option('useAutomationExtension', False)
        driver = webdriver.Firefox(firefox_profile= profile, options = firefox_options, executable_path=r'./geckodriver.exe')
        url = get_url_perfil(marca=perfil["marca"], ano_vehiculo=str(perfil["año"]), modelo=perfil["modelo"], \
                   version=perfil["version"], codigo= str(perfil["cp"]), edad=str(perfil["edad"]), sexo=str(perfil["sexo"]),driver=driver)
        urls[f"{perfil['id']}"] = url
    print(urls)
    df = pd.DataFrame(urls, index=["Seguros"] ).T.reset_index()
    df.rename(columns={"index":"Id"}, inplace=True)
    df["Fecha"] = datetime.date.today().strftime("%d-%m-%Y")
    df.to_excel("./urls.xlsx", index=False)
 
    return urls


In [54]:
%%time
get_all_perfiles()

Tiempo estimado total es: 10.0, minutos
*1 - ID, Obteniendo el perfil
[{'L/17': '23022'}, {'SS CONVERTIBLE         L/17': '44242'}]
 L/17
44242
tiempo de espera:  41.337737444242194

tiempo de espera:  59.142740147819026
*2 - ID, Obteniendo el perfil
[{'L/16 1.6 4 P S': '21942'}, {'L/16 2.0 5 P TITANIUM POWER': '21946'}, {'L/16 2.0 5 P SE': '21954'}, {'L/16 2.0 4 P SE PLUS': '21961'}, {'L/16 1.6 5 P S': '21966'}, {'L/16 2.0 4 P SE': '21975'}, {'L/16 2.0 4 P TITANIUM POWER': '21982'}, {'L/16 2.0 5 P SE PLUS': '21995'}, {'L/16 2.0 4 P SE PLUS POWER': '21996'}, {'L/16 2.0 5 P SE PLUS POWER': '22000'}, {'L/16 2.0 5 P TITANIUM': '22010'}, {'L/16 2.0 4 P TITANIUM': '22017'}, {'L/16 2.0 4 P SE AUT': '23247'}, {'L/16 2.0 5 P SE AUT': '23248'}]
L/16 1.6 4 P S
21942
tiempo de espera:  35


tiempo de espera:  67.29996436835086
*3 - ID, Obteniendo el perfil
[{'1.8 SE-G L/19 CVT HYBRID': '44394'}, {'2.0 SE-G L/19 CVT': '44395'}, {'2.0 XLI  L/20 CVT': '44586'}, {'1.8 XEI L/20 CVT HYBRID': '44587'}, 

{'1': 'https://www.comparaencasa.com/seguros-de-auto/resultados/chevrolet-en-camaro-l-17-capital-federal/eecd2aa202bc4e3fa71689dcdb4fcc16/',
 '2': 'https://www.comparaencasa.com/seguros-de-auto/resultados/ford-en-focus-l-16-1-6-4-p-s-capital-federal/40027a26dae54c03bcdfda1d7e006365/',
 '3': 'https://www.comparaencasa.com/seguros-de-auto/resultados/toyota-en-corolla-1-8-se-g-l-19-cvt-hybrid-capital-federal/608d1f84a6da40c28576bcb5d1ea8308/',
 '4': 'https://www.comparaencasa.com/seguros-de-auto/resultados/toyota-en-hilux-l-16-2-4-dc-4x2-tdi-sr-capital-federal/41478a85258a418a87b69ea4b79f1ffb/',
 '5': 'https://www.comparaencasa.com/seguros-de-auto/cotizador/?car_brand=28&year=2018&car_version=22722&car_model=ONIX',
 '6': 'https://www.comparaencasa.com/seguros-de-auto/resultados/chevrolet-en-camaro-l-17-capital-federal/f6233af347bd429b83ff134da7164d8e/',
 '7': 'https://www.comparaencasa.com/seguros-de-auto/resultados/ford-en-focus-l-16-1-6-4-p-s-capital-federal/995aedb8af634eb29a24a846cc69